### (Some) Problems with The Approach used in Assignment 1

* Lack of overlap between a large collection of documents
  * The more disparate documents we consider, the smaller the intersection
  * Can substantially hamper clustering and classification

* Considering all the non-stop words shared between the documents (union) leads to an unnecessarily lot of very large datasets
  * Many words that share a prefix, e.g., leave, leaving, left all lead to different counts despite potentially referencing the same concept

* The semantic meaning of words is not considered
  * We want the words Bank in the following sentences to have different meanings and hence counts. 
    * "She works at the bank across the street" vs. "houses on the bank of the river flooded due to. a storm surge"
  * We want the words Bank and financial institution to have the same meaning. E.g.:
    * "She works at the financial institution across the street" vs. "She's been working at the bank for over 5 years"
  * This is (was?) a *very hard* problem

### Distribution of Words in a Text 

* The frequency distribution of words in a language follows Zipf's law
  * Just FYI: this makes computign statistics rather difficult or impossible
  
![](https://www.dropbox.com/s/neydq8wi2kqqof3/zipf_law.png?dl=1)

### Document Similarity: Deriving A Matching Score

* Document Search in Information Retrieval (IR) is typically concerned with scoring matches based on some order of perceived importance=
  * Finding similar documents in a corpus
  * Ideally, we want the search to rank the hits (documents found) by their similarity to the query  
  We wish to return in order the documents most likely to be useful to the individual searching

* How can we rank-order the documents in the collection with respect to a query?
  * Assign a score – say in [0, 1] – to each document
  * This score measures how well hit and query “match”

* Document Retrieval is a field with a long history of creative solutions to similar problems



### Document Similarity: Deriving A Matching Score



* Consider a trivial search query consisting of a single term 

* If the query term does not occur in the document, score should be 0

* The more frequent the query term in the document, the higher the score (should be)




### Jaccard Coefficient


* Recall that the Jaccard Coefficient is a commonly used measure of overlap between two sets A and B
  * The number of overlaps between A and B normalized by all the words in A and B.
* Does not require A and B to have the same size.
* Always assigns a number between 0 and 1.

* Shortcoming:
* Does not consider term frequency
* Jaccard doesn’t consider the fact that rare terms in a collection are more informative than frequent terms. 
  * The reason why the intersection is not a good idea

* We need a more sophisticated way of normalizing for length (instead of $|A \cup B|$)




### Term-document Count Matrices

* In a count matrix, each word is represented with its frequency (count in a document)
  * As was mentioned in Assignment 2, this is called the bag of words model 
* Does not consider the order of words in the document

* `John is quicker than Mary` and `Mary is quicker than John` have the exact same vectors

### Term frequency `tf`

* Formally, the term frequency $tf_{t,d}$ of term $t$ in document $d$ is defined as the number of times that $t$ occurs in $d$.
*  The $tf$ is typically informative of the quality of a match, however, differences in $tf$ do not directly determine importance. E.g.:

  * A document with 10 occurrences of the term is more relevant than a document with 1 occurrence of the term, but not 10 times more relevant.

  * I.e., relevance does not increase proportionally with term frequency.




### Log-frequency weighting

* The log-frequency weight of term $t$ in $d$ is

$$
w_{t,d} = \left\{
    \begin{array}{ll}
        1+\log_{10}\mbox{tf}_{t,d} & \mbox{if } \mbox{tf}_{t,d} > 0\\
        0 & \mbox{otherwise}
    \end{array}
\right.
$$

```0 → 0, 1 → 1, 2 → 1.3, 10 → 2, 1000 → 4, etc.```

* Score for a document-query pair: sum over terms t in both q and d:

$$
\mbox{score} = \sum_{t\in \cap d}(1+\log_{10}\mbox{tf}_{t,d})
$$

* The score is 0 if none of the query terms is present in the document.

* Despite other variations, the essence of the calculations remains the same




### Document frequency

* We still have the issue of rare terms
  * Rare terms are more informative than frequent terms
    * Recall stop words
* Consider a term in the query that is rare in the collection (e.g., arachnocentric)
  * A document containing this term is very likely to be relevant to the query arachnocentric
  * The term is very likely to be relevant to clustering the docuements  
  * Thus, we want a high weight for rare terms like arachnocentric.


### Document Frequency Contd'

* Frequent terms are less informative than rare terms.
* Consider a query term that is frequent in the collection (e.g., `high`, `increase`, `true`)
  * Based on the $tf$ score alone, a document containing such a term is more likely to be relevant than a document that doesn’t
  * But it’s not a sure indicator of relevance.
* To assess the frequency of a work in a document, we'll compute (normlaize by) the document frequency `df` 


### `idf` Weight

$\mbox{df}_{t,d}$ frequency of $t$ in a single document $d$
* $\mbox{df}_t$ is an inverse measure of the informativeness of $t$
    * $\mbox{df}_t \le N$, where $N$ is the total number of documents.

* We define the $\mbox{idf}$ (inverse document frequency) of $t$ by
$$
idf_t = log_{10}(N/\mbox{df}_t)
$$
* We'd rather work with ther inverse to avoid very small numbers since $N>>\mbox{df}_f$
* We use the `log` to dampen the effect of idf
  * Particularly useful when we have a large number of documents



### `tf-idf` Weighting

* The `tf-idf` weight of a term is the product of its `tf` weight and its `idf` weight.

\begin{equation} \label{eq1}
\begin{split}
w_{t,d} &= \mbox{tf}_{t,d} \times \mbox{idf}_t\\
&= log(1+\mbox{tf}_{t,d}) \times log(N/\mbox{df}_t)
\end{split}
\end{equation}
* A popular weighting scheme in information retrieval
  * Alternative names: tf.idf, tf x idf

* Increases with the number of occurrences within a document
* Increases with the rareness of the term in the collection

### Score for a Document Given a Query


$$
Score(Q, T) = \sum_{t\in Q\cap T} \mbox{tf}.\mbox{idf}_{t,d}
$$

* There are many variants
  * How `tf` is computed (with/without logs)
  * Whether the terms in the query are also weighted




### Using `tf-idf` for Feature Engineering
* Each document is represented by a real-valued vector of $\mbox{tf-idf}$ weights $\in R^{|V|}$

![](https://www.dropbox.com/s/1bx77e488ee6wek/count_tf_idf.png?dl=1)

### Documents, Vectors and Space Proximity in IR


* **Key idea 1**: Do the same for queries: represent them as vectors in the space
* **Key idea 2**: Rank documents according to their proximity to the query in this space
  * proximity = similarity of vectors
    * Euclidean distance is a bad idea when the instances have differet lengths
      * The Euclidean distance is large for vectors of different lengths.
* **Key idea 3**: Rank documents according to angle with query.


In [21]:
doc_1 = """The king hath happily received, Macbeth,
The news of thy success; and when he reads
Thy personal venture in the rebels' fight,
His wonders and his praises do contend
Which should be thine or his: silenced with that,
In viewing o'er the rest o' the selfsame day,
He finds thee in the stout Norweyan ranks,
Nothing afeard of what thyself didst make,
Strange images of death. As thick as hail
Came post with post; and every one did bear
Thy praises in his kingdom's great defence,
And pour'd them down before him."""

doc_1 = doc_1.upper()

In [46]:
lorem_ipsum = """
Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do 
eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut 
enim ad minim veniam, quis nostrud exercitation ullamco laboris 
nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor 
in reprehenderit in voluptate velit esse cillum dolore eu fugiat
nulla pariatur.
""".upper()

In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
corpus = [doc_1, doc_1 + lorem_ipsum]
vocabulary = ['king', 'happily', 'and', "thy", "ipsum"]
c_vec = CountVectorizer(vocabulary=vocabulary)
tfidf_vec = TfidfVectorizer(vocabulary=vocabulary)



In [101]:
c_vec.fit_transform(corpus).todense()

matrix([[1, 1, 4, 3, 0],
        [1, 1, 4, 3, 1]])

In [102]:
tfidf_vec.fit_transform(corpus).todense()

matrix([[0.19245009, 0.19245009, 0.76980036, 0.57735027, 0.        ],
        [0.18577437, 0.18577437, 0.74309746, 0.5573231 , 0.26109939]])

### From Angles to Cosines

* In information retrieval, the following two notions are equivalent.
  * Rank documents in decreasing order of the angle between query and hit
  * Rank documents in increasing order of cosine(query,hit)

* Cosine is a monotonically decreasing function for the interval [0o, 180o]

![](https://www.dropbox.com/s/lpq4vvnlnmz0oxw/cosine.png?dl=1)

### Length Normalization

* A vector can be (length-) normalized by dividing each of its components by its length 
  * We commonly use the $L2$ norm:

* Dividing a vector by its $L2$ norm makes it a unit (length) vector

  * Effect on the two documents $d$ and $d′$ (d appended to itself) have identical vectors after length-normalization.
  * Thus, long and short documents now have comparable weights


### Cosine Similairity

* $q_i$ is the `tf-idf` weight of term `i` in the query
* $d_i$ is the `tf-idf` weight of term `i` in the document

![](https://www.dropbox.com/s/4x1fb50xiqidmnf/cos_equation.png?dl=1)

### Cosine Similarity Illustrated

![](https://www.dropbox.com/s/4inqt6nf9mfz6h9/cosine_similarity.png?dl=1)

### Example 
* Books: "Sense and Sensibility", "Pride and Prejudice", "Wuthering Heights?".

<img src="https://www.dropbox.com/s/z28xu8xxhuv8ll5/example_books.png?dl=1" alt="Drawing" style="width: 400px;"/>

```
cos(SaS,PaP) ≈ 0.789 × 0.832 + 0.515 × 0.555 + 0.335 × 0.0 + 0.0 × 0.0 ≈ 0.94
cos(SaS,WH) ≈ 0.79
cos(PaP,WH) ≈ 0.69
```

### `tf-idf` Weighing  Variants

* Just an FYI

<img src="https://www.dropbox.com/s/r88cmbmaqyk7hcp/weighting_schemes.png?dl=1" alt="Drawing" style="width: 400px;"/>

* identifies components of the SMART notation: combination in use in a search engine (ddd.qqq)
  * e.g.: lnc.ltc
  * ```To the legacy of the SMART system belongs the so-called SMART triple notation, a mnemonic scheme for denoting tf-idf weighting variants in the vector space model. The mnemonic for representing a combination of weights takes the form ddd.qqq, where the first three letters represents the term weighting of the collection document vector and the second three letters represents the term weighting for the query document vector. For example, ltc.lnn represents the ltc weighting applied to a collection document and the lnn weighting applied to a query document.```  https://en.wikipedia.org/wiki/SMART_Information_Retrieval_System
  
* See [Introduction to Information Retrieval](https://nlp.stanford.edu/IR-book/) for more info if you're interested in the topic


